<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html week6.do.txt --no_mako -->
<!-- dom:TITLE: February 27-March 3, 2023: Quantum Computing, Quantum Machine Learning and Quantum Information Theories -->

# February 27-March 3, 2023: Quantum Computing, Quantum Machine Learning and Quantum Information Theories
**Morten Hjorth-Jensen**, Department of Physics, University of Oslo and Department of Physics and Astronomy and National Superconducting Cyclotron Laboratory, Michigan State University

Date: **Jan 19, 2023**

Copyright 1999-2023, Morten Hjorth-Jensen. Released under CC Attribution-NonCommercial 4.0 license

## Quantum Fourier Transform and Hamiltonians

1. Quantum Fourier Transforms

2. Quantum Phase estimation algorithm

3. Eigenstates of specific Hamiltonians